# 1.0 Install Packages and Import Libraries

In [1]:
!pip install -q -U bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl rouge_score wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependenc

In [2]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)
from tqdm import tqdm
from trl import SFTTrainer
import torch
import time
import pandas as pd
import numpy as np
from huggingface_hub import interpreter_login
from huggingface_hub import login
import wandb
import os

# Login to HuggingFace
# # interpreter_login()

# Login to Huggingface
api_token = "<API KEY HERE>"
login(token=api_token)

In [3]:
# # Login to weights and biases (to track training metrics)
# wandb.login()
# %env WANDB_PROJECT=Fine-Tune-QLoRA

#------------------------------------------------------------------------------------------------------

# Set your W&B API key here
os.environ["WANDB_API_KEY"] = "<API KEY HERE>"

# Log in to W&B
wandb.login()

# Set your W&B project
os.environ["WANDB_PROJECT"] = "Fine-Tune-QLoRA"

print("Successfully logged into Weights & Biases!")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: 0132114 (0132114-uow-malaysia). Use `wandb login --relogin` to force relogin


Successfully logged into Weights & Biases!


# 2.0 Load Dataset

In [4]:
from datasets import load_dataset
from datasets.dataset_dict import DatasetDict

# Load the dataset
dataset = load_dataset("Amod/mental_health_counseling_conversations")

# Remove 80% of the dataset (reduce dataset size)
reduced_dataset = dataset['train'].shuffle(seed=42).select(range(int(len(dataset['train']) * 0.2)))

# Split the remaining 20% into train, test, and validation sets
train_testvalid = reduced_dataset.train_test_split(test_size=0.2, seed=42)  # 80% train, 20% test+valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42)  # 10% test, 10% valid

# Combine into a DatasetDict
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']
})

# Check the size of the splits
print(f"Train size: {len(dataset['train'])}")
print(f"Test size: {len(dataset['test'])}")
print(f"Validation size: {len(dataset['validation'])}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.82k [00:00<?, ?B/s]

combined_dataset.json:   0%|          | 0.00/4.79M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

Train size: 561
Test size: 71
Validation size: 70


In [ ]:
# from datasets import load_dataset
# from datasets.dataset_dict import DatasetDict

# # Load the dataset
# dataset = load_dataset("Amod/mental_health_counseling_conversations")

# # Perform the splits
# train_testvalid = dataset['train'].train_test_split(test_size=0.2, seed=42)  # 80% train, 20% test+valid
# test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42)  # 10% test, 10% valid

# # Combine into a DatasetDict
# dataset = DatasetDict({
#     'train': train_testvalid['train'],
#     'test': test_valid['test'],
#     'validation': test_valid['train']
# })

# # Check the size of the splits
# print(f"Train size: {len(dataset['train'])}")
# print(f"Test size: {len(dataset['test'])}")
# print(f"Validation size: {len(dataset['validation'])}")


Train size: 2809
Test size: 352
Validation size: 351


In [5]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response'],
        num_rows: 561
    })
    test: Dataset({
        features: ['Context', 'Response'],
        num_rows: 71
    })
    validation: Dataset({
        features: ['Context', 'Response'],
        num_rows: 70
    })
})


In [6]:
print(dataset['train'][0])

{'Context': 'Something happened this summer that I cannot forgive myself for. When I think about what happened, I feel ashamed and guilty even though my loved ones forgave me.', 'Response': "What a beautiful discussion of guilt and shame Lynda offers! Those\ntwo demons start out as pesky and little when we are young, then grow with us to\nbecome quite formidable. I'd like to throw in a technique I learned from Randy\nRausch (author of\xa0The Last Lecture)\xa0called The 3-Step Apology.\nI teach this important bit of wisdom to my clients when they need to apologize\nto someone else but more importantly when they need to forgive themselves.Yes, when we have trouble forgiving ourselves we sometimes forget\nthat there is part of us that just doesn't know how to apologize effectively.\nHere goes:Step One: Say what you did and the damage it caused.Step Two: Say you're sorry and accept full\nresponsibility (no excuses, no explanation, no defensiveness: “It was all my\nfault.”)Step\nThree: Ask 

# 3.0 Configure Bits and Bytes
- allows model to be loaded in quantized/double quantized form
- quantized: represent a given set of numbers with less info (e.g. 1.5 --> 1)


In [7]:
# Ensure the computation uses 16-bit floating-point (reduce memory usage, speed up training)
compute_dtype = getattr(torch, "float16")

# Configure Bits and Bytes to load the model in 4-bit (quantized)
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, # Load the weights in 4 bit
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype, # Uses 16-bit floating-point (float16)
        bnb_4bit_use_double_quant=True, # Enable double quantization
    )

# 4.0 Load the Pretrained Model in 4-bit (Quantized)

In [8]:
# Load the pretrained model, 'meta-llama/Llama-3.2-1B-Instruct' required authorization
base_model_name = 'Qwen/Qwen2.5-0.5B-Instruct'
device_map = "auto" #{"": 0}
base_model = AutoModelForCausalLM.from_pretrained(base_model_name,
                                                      device_map=device_map,
                                                      quantization_config=bnb_config, # To load in 4-bit and double quantization
                                                      trust_remote_code=True,
                                                      use_cache = False,
                                                      use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

# 5.0 Configure the Tokenizer

In [9]:
# Configure the tokenizer, use left-padding to optimize memory usage during training.
tokenizer = AutoTokenizer.from_pretrained(base_model_name,
                                          trust_remote_code=True,
                                          padding_side="left",
                                          add_eos_token=True,
                                          add_bos_token=True,
                                          use_fast=False)

tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

# 6.0 Test the Base Model's Response

In [10]:
# Insert prompt
prompt = "Something happened this summer that I cannot forgive myself for. When I think about what happened, I feel ashamed and guilty even though my loved ones forgave me."

# Format the prompt
messages = [
    {"role": "system", "content": "You are a helpful mental health therapist. Provide responses that can help the users."},
    {"role": "user", "content": prompt}
]

# Apply chat template
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False, # Keep text as string
    add_generation_prompt=True # Adds additional instructions (if needed)
)

# Tokenize the text
model_inputs = tokenizer([text], return_tensors="pt").to(base_model.device)

# Generate response
generated_ids = base_model.generate(
    **model_inputs,
    max_new_tokens=512
)

# Get the generated tokens
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

# Decode the tokens into text
base_response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("BASE MODEL RESPONSE \n============================================== \n", base_response)

BASE MODEL RESPONSE 
 Getting to a place where you feel content and at peace with yourself is a journey, but there are several steps you can take to help you achieve this:

1. **Self-Reflection**: Spend time reflecting on your thoughts, feelings, and experiences in your life. This helps you understand what you want out of life.

2. **Set Realistic Goals**: Identify what you want to achieve. Set goals based on what makes you happy or fulfilled, such as having enough sleep, spending quality time with loved ones, or pursuing personal growth.

3. **Practice Self-Care**: Regularly engage in activities that nourish your body and mind, including exercise, meditation, and adequate rest.

4. **Establish Routine**: Establishing routines can provide structure and stability, making it easier to maintain a sense of contentment.

5. **Mindfulness**: Practice mindfulness techniques like meditation, deep breathing, or yoga to manage stress and anxiety.

6. **Seek Support**: Surround yourself with peop

# 7.0 Dataset Preprocessing

## 7.1 Define Preprocessing Functions

### 7.1.1 Change Prompt Format in Each Row

In [11]:
from functools import partial

def create_prompt_formats(sample):
    # Define the instruction
    instruction = "Below is a question regarding the user and its corresponding response that helps the user with their mental health."

    # Format the question and answer into the desired prompt
    ques_prompt = f"<QUESTION>: {sample['Context']}"
    ans_response = f"<ANSWER>: {sample['Response']}"

    # Combine the instruction, question, and answer into a single formatted text
    formatted_prompt = f"{instruction}\n{ques_prompt}\n{ans_response}"

    # Add the new 'text' key to the sample dictionary
    sample["text"] = formatted_prompt

    return sample

### 7.1.2 Get the Model's Maximum Token Limit and Tokenize Each Batch


In [12]:
# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
# To get the model's maximum token limit
def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length

# Tokenize each batch
def preprocess_batch(batch, tokenizer, max_length):
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )

### 7.1.3 Process Entire Dataset
- combines previous functions
- processes entire dataset

In [13]:
# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
# To process the entire dataset
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed, dataset):
    # Add prompt to each sample
    dataset = dataset.map(create_prompt_formats)#, batched=True)

    # Apply preprocessing to each batch of the dataset & and remove existing fields 'question' and 'answer'
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=['Context', 'Response'],
    )

    # Filter out rows with input_ids exceeding max_length
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)

    # Shuffle dataset
    dataset = dataset.shuffle(seed=seed)

    return dataset

## 7.2 Start Processing
- process the training and validation dataset

In [14]:
# Set a random seed
# %%time
from transformers import set_seed
seed = 42
set_seed(seed)

# Get the max_length
max_length = get_max_length(base_model)
print(max_length)

# Preprocess the train and validation dataset
train_dataset = preprocess_dataset(tokenizer, max_length, seed, dataset['train'])
val_dataset = preprocess_dataset(tokenizer, max_length, seed, dataset['validation'])

Found max lenth: 32768
32768


Map:   0%|          | 0/561 [00:00<?, ? examples/s]

Map:   0%|          | 0/561 [00:00<?, ? examples/s]

Filter:   0%|          | 0/561 [00:00<?, ? examples/s]

Map:   0%|          | 0/70 [00:00<?, ? examples/s]

Map:   0%|          | 0/70 [00:00<?, ? examples/s]

Filter:   0%|          | 0/70 [00:00<?, ? examples/s]

# 8.0 Save the Processed Datasets

In [15]:
from google.colab import drive
drive.mount('/content/drive')

# Define paths for saving datasets
train_save_path = '/content/drive/My Drive/hf_train_dataset'
val_save_path = '/content/drive/My Drive/hf_val_dataset'

# Save the datasets
train_dataset.save_to_disk(train_save_path)
val_dataset.save_to_disk(val_save_path)

print(f"Train dataset saved to {train_save_path}")
print(f"Validation dataset saved to {val_save_path}")


Mounted at /content/drive


Saving the dataset (0/1 shards):   0%|          | 0/561 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/70 [00:00<?, ? examples/s]

Train dataset saved to /content/drive/My Drive/hf_train_dataset
Validation dataset saved to /content/drive/My Drive/hf_val_dataset


In [16]:
print(train_dataset)

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 561
})
